In [1]:
import requests             
from bs4 import BeautifulSoup 
import csv                  
import webbrowser
import io
import pandas as pd

In [2]:
def display(content, filename='output.html'):
    with open(filename, 'wb') as f:
        f.write(content)
        webbrowser.open(filename)

def get_soup(session, url, show=False):
    r = session.get(url)
    if show:
        display(r.content, 'temp.html')

    if r.status_code != 200: # not OK
        print('[get_soup] status code:', r.status_code)
    else:
        return BeautifulSoup(r.text, 'html.parser')
    
def post_soup(session, url, params, show=False):
    '''Read HTML from server and convert to Soup'''

    r = session.post(url, data=params)
    
    if show:
        display(r.content, 'temp.html')

    if r.status_code != 200: # not OK
        print('[post_soup] status code:', r.status_code)
    else:
        return BeautifulSoup(r.text, 'html.parser')

In [3]:
def scrape(url, lang='ALL'):

    # create session to keep all cookies (etc.) between requests
    session = requests.Session()

    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:57.0) Gecko/20100101 Firefox/57.0',
    })


    items = parse(session, url + '?filterLang=' + lang)

    return items

In [4]:
def parse(session, url):
    '''Get number of reviews and start getting subpages with reviews'''

    print('[parse] url:', url)

    soup = get_soup(session, url, show = True)

    if not soup:
        print('[parse] no soup:', url)
        return
    
#     print(soup.find("span", {"class": "reviewCount ui_link level_4"}))

#     num_reviews = soup.find('span', attrs = {'class' : 'reviewCount ui_link level_4'}).text # get text
#     num_reviews = num_reviews.split()[0]
#     num_reviews = int(num_reviews) # convert text into integer
#     print('[parse] num_reviews ALL:', num_reviews)

#     url_template = 'https://www.tripadvisor.in/Hotel_Review-or{}-g297697-d2437405-Reviews-Grand_Ixora_Kuta_Resort-Kuta_Kuta_District_Bali.html#REVIEWS'
#     print('[parse] url_template:', url_template)
    

    items = []
    curr_pg_items = parse_reviews(session, url)
    if curr_pg_items:
        items += curr_pg_items
#     offset = 5

#     while(True):
#         subpage_url = url_template.format(offset)

#         subpage_items = parse_reviews(session, subpage_url)
#         if not subpage_items:
#             break

#         items += subpage_items

#         if len(subpage_items) < 5:
#             break

#         offset += 5

    return items

In [5]:
def get_reviews_ids(soup):

    items = soup.find_all('div', attrs={'data-reviewid': True})

    if items:
        reviews_ids = [x.attrs['data-reviewid'] for x in items]
        print('[get_reviews_ids] data-reviewid:', reviews_ids)
        return reviews_ids
    
def get_more(session, reviews_ids):

    url = 'https://www.tripadvisor.com/OverlayWidgetAjax?Mode=EXPANDED_HOTEL_REVIEWS_RESP&metaReferer=Hotel_Review'

    payload = {
        'reviews': ','.join(reviews_ids), # ie. "577882734,577547902,577300887",
        #'contextChoice': 'DETAIL_HR', # ???
        'widgetChoice': 'EXPANDED_HOTEL_REVIEW_HSX', # ???
        'haveJses': 'earlyRequireDefine,amdearly,global_error,long_lived_global,apg-Hotel_Review,apg-Hotel_Review-in,bootstrap,desktop-rooms-guests-dust-en_US,responsive-calendar-templates-dust-en_US,taevents',
        'haveCsses': 'apg-Hotel_Review-in',
        'Action': 'install',
    }

    soup = post_soup(session, url, payload)

    return soup

def parse_reviews(session, url):
    '''Get all reviews from one page'''
    global greview

    print('[parse_reviews] url:', url)

    soup =  get_soup(session, url)

    if not soup:
        print('[parse_reviews] no soup:', url)
        return

    hotel_name = soup.find('h1', id='HEADING').text

    reviews_ids = get_reviews_ids(soup)
    if not reviews_ids:
        return

    soup = get_more(session, reviews_ids)

    if not soup:
        print('[parse_reviews] no soup:', url)
        return

    items = []

    for idx, review in enumerate(soup.find_all('div', class_='reviewSelector')):
#         print(review)
        greview = review
        badgets = review.find_all('span', class_='badgetext')
        if len(badgets) > 0:
            contributions = badgets[0].text
        else:
            contributions = '0'

        if len(badgets) > 1:
            helpful_vote = badgets[1].text
        else:
            helpful_vote = '0'
        user_loc = review.select_one('div.userLoc strong')
        if user_loc:
            user_loc = user_loc.text
        else:
            user_loc = ''
            
        bubble_rating = review.select_one('span.ui_bubble_rating')['class']
        bubble_rating = bubble_rating[1].split('_')[-1]
        
        gg_triptype = review.find('div', class_='prw_rup prw_reviews_category_ratings_hsx')
        if  gg_triptype is not None and len(gg_triptype.text.split(':')) > 1:
            trip_type = gg_triptype.text.split(':')[1]
        else:
            trip_type = ''
        
        gg_mos = soup.find('div', attrs = {'class' : 'prw_rup prw_reviews_stay_date_hsx'})
        if gg_mos and len(gg_mos.text.split()) > 1:
            month_of_stay = gg_mos.text.split()[-2]
        else:
            month_of_stay = 'Not Mentioned'
            
        gg_reviewtitle = review.find('div', class_='quote')
        if gg_reviewtitle:
            reviews_title = gg_reviewtitle.text
        else:
            reviews_title = 'NA'

        item = {
            'review_body': review.find('p', class_='partial_entry').text,
            'review_date': review.find('span', class_='ratingDate')['title'], # 'ratingDate' instead of 'relativeDate'
            'reviews_title':reviews_title,
            'rating': bubble_rating,
            'user location': user_loc,
            'trip_type': trip_type,
            'month_of_stay':month_of_stay,
            'helpful_vote':helpful_vote,
            'hotel_name':hotel_name,
            'contributions': contributions
        }

        items.append(item)
        print('\n--- review ---\n')
        for key,val in item.items():
            print(' ', key, ':', val)

    print()

    return items

def write_in_csv(items, filename='results.csv',
                  headers=['review title', 'review body',
                           'review date', 'contributions', 'helpful vote',
                           'user name' , 'user location', 'rating','reviews_title'],
                  mode='w'):

    print('--- CSV ---')

    with io.open(filename, mode, encoding="utf-8") as csvfile:
        csv_file = csv.DictWriter(csvfile, headers)

        if mode == 'w':
            csv_file.writeheader()

        csv_file.writerows(items)

In [6]:
def make_rows(items, rows):
    for d in items: 
        curr_row = [d['review_body'], d['review_date'], d['reviews_title'],d['rating'], d['trip_type'], d['user location'], d['month_of_stay'], 
                   d['contributions'], d['helpful_vote'], d['hotel_name']]
        rows.append(curr_row)
    return rows

## Put Start_url in this Block

In [7]:
# DB_COLUMN   = 'review_body'
# DB_COLUMN1 = 'review_date'
#https://www.tripadvisor.in/Hotel_Review-g297697-d2437405-Reviews-or465-Grand_Ixora_Kuta_Resort-Kuta_Kuta_District_Bali.html#REVIEWS
start_url = 'https://www.tripadvisor.in/Hotel_Review-or{}-g297697-d2437405-Reviews-Grand_Ixora_Kuta_Resort-Kuta_Kuta_District_Bali.html#REVIEWS'
x = 'https://www.tripadvisor.in/Hotel_Review-or'
y = '-g297697-d2437405-Reviews-Grand_Ixora_Kuta_Resort-Kuta_Kuta_District_Bali.html#REVIEWS'
lang = 'en'

headers = [ 
    'review_body', 
    'review_date', 
    'reviews_title',
    'rating',
    'trip_type',
    'user location',
    'month_of_stay',
    'contributions',
    'helpful_vote',
    'hotel_name' ]

# for url in start_urls:
final_rows =[]
for i in range(5, 496, 5):
#     if i > 10:
#         break
    url = x + str(i) + y
    print(url)
    # get all reviews for 'url' and 'lang'
    items = scrape(url, lang)
    final_rows = make_rows(items, final_rows)


https://www.tripadvisor.in/Hotel_Review-or5-g297697-d2437405-Reviews-Grand_Ixora_Kuta_Resort-Kuta_Kuta_District_Bali.html#REVIEWS
[parse] url: https://www.tripadvisor.in/Hotel_Review-or5-g297697-d2437405-Reviews-Grand_Ixora_Kuta_Resort-Kuta_Kuta_District_Bali.html#REVIEWS?filterLang=en
[parse_reviews] url: https://www.tripadvisor.in/Hotel_Review-or5-g297697-d2437405-Reviews-Grand_Ixora_Kuta_Resort-Kuta_Kuta_District_Bali.html#REVIEWS?filterLang=en
[get_reviews_ids] data-reviewid: ['670058582', '669317567', '669288882', '666653169', '663869761']

--- review ---

  review_body : This hotel located in great area ... near almost every destination and attractionRoom spacy, bathroom with shower (no handshower) and aminities is good, coffee bar for free but air-con need more maintain also bedcover smell a bitBreakfast great and staff service's very nice and helpful
  review_date : April 28, 2019
  reviews_title : Good deal hotel at great location
  rating : 30
  user location : Jakarta, Indon

[parse_reviews] url: https://www.tripadvisor.in/Hotel_Review-or15-g297697-d2437405-Reviews-Grand_Ixora_Kuta_Resort-Kuta_Kuta_District_Bali.html#REVIEWS?filterLang=en
[get_reviews_ids] data-reviewid: ['649250940', '648244842', '647692731', '646321861', '643776099']

--- review ---

  review_body : We had all you can eat Breakfast here on our 1st Morning on recent trip to Bali whilst staying at another place that seemed very high.$ 7AUD plus tax was awesome for 5 people, very comfortable and staff were amazing. We later hired a Driver from Ixora Sodik who we did some great tours of regional Bali with. It wasn't a hotel listed on Booking.com so we stayed elsewhere I'd definitely go back here to either eat or stay of use Sodik's services.
  review_date : January 30, 2019
  reviews_title : Greatest all you can eat Breakfast in Kuta/Tuban area
  rating : 50
  user location : 
  trip_type :  Traveled with familyValueCleanlinessService
  month_of_stay : January
  helpful_vote : 22
  hotel_name

[get_reviews_ids] data-reviewid: ['628873881', '627716713', '627414705', '627411129', '627408443']

--- review ---

  review_body : I stayed here for 3 D/N with friends and familyThe location of hotel is superb very near to mall/market and food joints!Food is nice and widespread ,Pool is nice too.Staffs are very courteous and friendly.The only thing which I believe is negative is rooms are not that big....but thye are Ok.
  review_date : October 27, 2018
  reviews_title : Bali trip Oct2018
  rating : 40
  user location : Hyderabad, India
  trip_type :  Traveled with friendsLocationSleep QualityService
  month_of_stay : October
  helpful_vote : 2
  hotel_name : Grand Ixora Kuta Resort
  contributions : 4

--- review ---

  review_body : about this hotel the best thing is there wifi, its simply awesome, the speed the connectivity. and the worst thing about this hotel is there room service, rest everything was fine, nice breakfast, nice rooms, cleanliness was good, near to kuta beach and 

[parse_reviews] url: https://www.tripadvisor.in/Hotel_Review-or40-g297697-d2437405-Reviews-Grand_Ixora_Kuta_Resort-Kuta_Kuta_District_Bali.html#REVIEWS?filterLang=en
[get_reviews_ids] data-reviewid: ['621644216', '619619234', '618217016', '617165269', '616333130']

--- review ---

  review_body : We recently stayed here for a couple of days before we flew home as it is close to the airport. A short walk to Lippo mall and a bit further to discovery mall. The food was nice from the hotel restaurant but a little bit expensive. Rooms are small and we found that our bed linen would feel damp if we didn't have the air conditioner on. There was no where to sit on the balcony. The pool was good but it is under construction on one side. The location and price is good but for a couple of nights at most
  review_date : October 2, 2018
  reviews_title : Good location
  rating : 30
  user location : Sydney, Australia
  trip_type :  Traveled with family
  month_of_stay : September
  helpful_vote : 1

[parse_reviews] url: https://www.tripadvisor.in/Hotel_Review-or50-g297697-d2437405-Reviews-Grand_Ixora_Kuta_Resort-Kuta_Kuta_District_Bali.html#REVIEWS?filterLang=en
[get_reviews_ids] data-reviewid: ['606943192', '600659523', '599728519', '595707803', '592616858']

--- review ---

  review_body : Everything was convenient in terms of price and service .very friendly staff ..The hotel was close to the beach close to the shopping center and the supermarketStayed there for 18 days and frankly the hotel is magnificence i loved it was so nice the best hotel in bali so try it and see your self .you will be happy with mr andhyka thanksabbas sabtie ..16\8\2018australia
  review_date : August 16, 2018
  reviews_title : mr andhyka lovely
  rating : 50
  user location : 
  trip_type :  Traveled with friendsValueRoomsService
  month_of_stay : July
  helpful_vote : 0
  hotel_name : Grand Ixora Kuta Resort
  contributions : 1

--- review ---

  review_body : The hotel location is really good and the

...

[parse_reviews] url: https://www.tripadvisor.in/Hotel_Review-or425-g297697-d2437405-Reviews-Grand_Ixora_Kuta_Resort-Kuta_Kuta_District_Bali.html#REVIEWS?filterLang=en
[get_reviews_ids] data-reviewid: ['147281049', '147191845', '146532311', '146455653', '146092394']

--- review ---

  review_body : Stayed during early Dec for 7 days, with our son. Very NICE CLEAN hotel located right on Karthika Plaza road (this is the main road in KUTA), extremely near to the airport (about 5-6 minutes by car). Opt for a pool facing-highfloor room (the hotel has Lobby + 4 levels). We stayed in # 5112 (Superior Room) overlooking the pool - Just perfect. Upside- Centrally located @ Kuta.- close proximity to Airport (5-6 minutes by cab). Walking distance to Kuta beach, Discovery mall, Water Bom (water amusement park & rides), - the hotel provides complimentary bicycles (they have a total of 3 bicyles provided free on 1st come basis)- pool is just perfect not too huge- good for lazying around. Pool towels a

[parse_reviews] url: https://www.tripadvisor.in/Hotel_Review-or435-g297697-d2437405-Reviews-Grand_Ixora_Kuta_Resort-Kuta_Kuta_District_Bali.html#REVIEWS?filterLang=en
[get_reviews_ids] data-reviewid: ['144192707', '144114190', '143810395', '143572889', '143509798']

--- review ---

  review_body : I seen so many good reviews on here and had my hopes up. I realise now they offer you a souviner if you put a review on whilst you are there, so most would say positive things. Im going to let you know what i wish i knew before staying. Firstly the service and staff were amazing, no complaints. The room was small, and after ten days felt like a prision and the view outside my window was awful. The bathroom was tiny. The bed was like laying on a bed frame, extremely hard. Very small tv. There were ants everywhere in our room and i was worried about customs as i am sure there were ants in our bags.The pool is half the size you imagine it to be. I didnt like the location of it as the many touris

[parse_reviews] url: https://www.tripadvisor.in/Hotel_Review-or445-g297697-d2437405-Reviews-Grand_Ixora_Kuta_Resort-Kuta_Kuta_District_Bali.html#REVIEWS?filterLang=en
[get_reviews_ids] data-reviewid: ['141073753', '141073443', '141058314', '140929464', '140642240']

--- review ---

  review_body : Fantastic location, so close to Discovery Mall and everything you need in Kuta. The staff were extremely friendly and provided great service, our rooms were very comfortable and always made up nicely. My friends and I enjoyed using the spa for traditional Balanese massages, and were set up from day 1 with a very helpful driver who took us to some of Bali's greatest locations. Overall we had a very enjoyable stay at the Grand Whiz Hotel Kuta
  review_date : September 24, 2012
  reviews_title : Lovely Hotel
  rating : 50
  user location : Perth, Australia
  trip_type :  Traveled with friendsValueLocationSleep QualityRoomsCleanlinessService
  month_of_stay : September
  helpful_vote : 0
  hotel_

[parse_reviews] url: https://www.tripadvisor.in/Hotel_Review-or455-g297697-d2437405-Reviews-Grand_Ixora_Kuta_Resort-Kuta_Kuta_District_Bali.html#REVIEWS?filterLang=en
[get_reviews_ids] data-reviewid: ['137998610', '137532993', '136978633', '135866426', '135546460']

--- review ---

  review_body : I thought this is a good hotel because when I see the photos on the internet, it looks like a big hotel that have spacious rooms. Unfortunately, when I stay in superior rooms, the size of the room is quite similar to the standard room in other 3 star hotels. Also the size of the bathroom is too small for a 3 star hotel. The room rate per night is also expensive I think. It is like a rate in the 4 star hotel. The location is also quite far from Kuta beach. It takes 15-20 minutes to walk to Kuta beach. The closest one is the Discovery Mall and Waterboom park. It only takes 5-10 minutes walking to reach these two places. The staffs are not bad, but the receptionist are not too welcome to the loc

[parse_reviews] url: https://www.tripadvisor.in/Hotel_Review-or465-g297697-d2437405-Reviews-Grand_Ixora_Kuta_Resort-Kuta_Kuta_District_Bali.html#REVIEWS?filterLang=en
[get_reviews_ids] data-reviewid: ['125344138', '125076384', '123888369']

--- review ---

  review_body : This is a new hotel with small but good rooms,lovely staff and close enough to the action. Overall a good hotel and well priced.
  review_date : February 28, 2012
  reviews_title : Well placed and good value
  rating : 40
  user location : Sydney, Australia
  trip_type : 
  month_of_stay : Not Mentioned
  helpful_vote : 308
  hotel_name : Grand Ixora Kuta Resort
  contributions : 295

--- review ---

  review_body : This newly open hotel in Bali Kuta area is highly recommened. It is located far away enough from the busy and noisy area yet close enough, just 5-10 minutes walk away from shopping mall and Kuta shopping district. The swimming pool and rooms in this hotel is very clean and gives you a very cosy feeling sta

In [8]:
df = pd.DataFrame(data = final_rows, columns=headers)

In [9]:
df.shape

(478, 10)

In [12]:
df.to_csv('final_csv_478.csv', index= False)

In [10]:
END

NameError: name 'END' is not defined

#### Random Testing

In [ ]:
headers = [ 
    'review_body', 
    'review_date', 
    'rating',
    'trip_type',
    'user location',
    'month_of_stay',
    'contributions',
    'helpful_vote',
    'hotel_name' ]

lang = 'en'
st_url = 'https://www.tripadvisor.in/Hotel_Review-g297697-d2437405-Reviews-Grand_Ixora_Kuta_Resort-Kuta_Kuta_District_Bali.html#REVIEWS'
items = scrape(st_url, lang)

if not items:
    print('No reviews')
else:
    # write in CSV
    filename = st_url.split('Reviews-')[1][:-5] + '__' + lang
    print('filename:', filename)
    write_in_csv(items, filename + '.csv',headers=headers, mode='w')

In [ ]:
session = requests.Session()

session.headers.update({
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:57.0) Gecko/20100101 Firefox/57.0',
})
soup = get_soup(session, st_url, show = True)

if not soup:
    print('[parse] no soup:', url)
